微调（或迁移学习）是将在大型数据集上训练的模型调整为您的特定（可能小得多）数据集的过程。

# 术语

以下是您需要熟悉的常用术语：

|术语|定义|
|----|----|
|微调(Finetuning)|将在大型数据集上训练的模型调整为您的特定（可能更小）数据集的过程|
|迁移学习（Transfer learning）|微调的通用名称|
|骨干(Backbone)|在不同数据集上预训练的神经网络|
|头（Head）|另一个神经网络（通常较小）将主干映射到您的特定数据集|
|冻结(Freeze)|禁用模型的梯度更新（即：不学习）|
|解冻(Unfreeze)|启用模型的梯度更新|

# 在 Flash 中进行微调

从快速入门指南。

要使用任务进行微调：
* 使用为任务定制的 DataModule 加载您的数据并组织它（例如：ImageClassificationData）。
* 选择并初始化您的任务，其中内置了最先进的主干（例如：ImageClassifier）。
* 初始化 flash.core.trainer.Trainer。
* 选择微调策略（例如：“冻结”）并使用您的数据调用 flash.core.trainer.Trainer.finetune()。
* 保存您的微调模型。

这是一个微调的例子。

In [ ]:
from pytorch_lightning import seed_everything

import flash
from flash.core.classification import Labels
from flash.core.data.utils import download_data
from flash.image import ImageClassificationData, ImageClassifier

# set the random seeds.
seed_everything(42)

# 1. 下载和组织数据
download_data("https://pl-flash-data.s3.amazonaws.com/hymenoptera_data.zip", "data/")

datamodule = ImageClassificationData.from_folders(
    train_folder="data/hymenoptera_data/train/",
    val_folder="data/hymenoptera_data/val/",
    test_folder="data/hymenoptera_data/test/",
)

# 2. 使用所需的任务构建模型
model = ImageClassifier(backbone="resnet18", num_classes=datamodule.num_classes)

# 3. 创建训练器（运行一个 epoch 进行演示）
trainer = flash.Trainer(max_epochs=1, gpus=torch.cuda.device_count())

# 4. 微调模型
trainer.finetune(model, datamodule=datamodule, strategy="freeze")

# 5. 保存模型！
trainer.save_checkpoint("image_classification_model.pt")

# 使用微调模型

微调后，使用模型进行预测：

In [ ]:
# Serialize predictions as labels, automatically inferred from the training data in part 2.
model.serializer = Labels()

predictions = model.predict(
    [
        "data/hymenoptera_data/val/bees/65038344_52a45d090d.jpg",
        "data/hymenoptera_data/val/ants/2255445811_dabcdf7258.jpg",
    ]
)
print(predictions)

我们得到以下输出：

In [ ]:
['bees', 'ants']

或者您可以在任何地方使用保存的模型进行预测！

In [ ]:
from flash.image import ImageClassifier

# load finetuned checkpoint
model = ImageClassifier.load_from_checkpoint("image_classification_model.pt")

predictions = model.predict("path/to/your/own/image.png")

# 微调策略

微调是非常特定于任务的。 每个任务都编码了该任务的最佳微调实践。 但是，Flash 为您提供了一些默认的微调策略。

微调对两件事进行操作，模型主干和头部。 主干是预先训练好的神经网络。 头部是另一个神经网络，它在主干和您的特定数据集之间架起桥梁。

## 不冻结

在这个策略中，脊椎和头部从一开始就解冻了。

In [ ]:
trainer.finetune(model, datamodule, strategy="no_freeze")

在伪代码中，这看起来像：

In [ ]:
backbone = Resnet50()
head = nn.Linear(...)

backbone.unfreeze()
head.unfreeze()

train(backbone, head)

## 冻结

冻结策略始终保持主干冻结。

In [ ]:
trainer.finetune(model, datamodule, strategy="freeze")

伪代码如下所示：

In [ ]:
backbone = Resnet50()
head = nn.Linear(...)

# freeze backbone
backbone.freeze()
head.unfreeze()

train(backbone, head)

# 高级策略
每个微调策略也可以定制。
## 冻结解冻
默认情况下，在此策略中，主干被冻结 5 个时期，然后解冻：

In [ ]:
trainer.finetune(model, datamodule, strategy="freeze_unfreeze")

或者我们可以自定义它在不同的周期后解冻主干。 例如，要在 epoch 7 之后解冻：

In [ ]:
from flash.core.finetuning import FreezeUnfreeze

trainer.finetune(model, datamodule, strategy=FreezeUnfreeze(unfreeze_epoch=7))

在幕后，伪代码如下所示：

In [ ]:
backbone = Resnet50()
head = nn.Linear(...)

# freeze backbone
backbone.freeze()
head.unfreeze()

train(backbone, head, epochs=10)

# unfreeze after 10 epochs
backbone.unfreeze()

train(backbone, head)

## unfreeze_milestones

此策略允许您以预定的时间间隔解冻部分主干

这是一个示例，其中： - 主干开始冻结 - 在 epoch 3 最后 2 层解冻 - 在 epoch 8 时，完整的主干解冻

In [ ]:
from flash.core.finetuning import UnfreezeMilestones

trainer.finetune(model, datamodule, strategy=UnfreezeMilestones(unfreeze_milestones=(3, 8), num_layers=2))

在幕后，伪代码如下所示：

In [ ]:
backbone = Resnet50()
head = nn.Linear(...)

# freeze backbone
backbone.freeze()
head.unfreeze()

train(backbone, head, epochs=3)

# unfreeze last 2 layers at epoch 3
backbone.unfreeze_last_layers(2)

train(backbone, head, epochs=8)

# unfreeze the full backbone
backbone.unfreeze()

# 自定义策略

如需更多自定义，请创建您自己的微调回调。 在此处了解有关回调的更多信息。

In [ ]:
from flash.core.finetuning import FlashBaseFinetuning

# 创建微调回调
class FeatureExtractorFreezeUnfreeze(FlashBaseFinetuning):
    def __init__(self, unfreeze_epoch: int = 5, train_bn: bool = True):
        # this will set self.attr_names as ["backbone"]
        super().__init__("backbone", train_bn)
        self._unfreeze_epoch = unfreeze_epoch

    def finetune_function(self, pl_module, current_epoch, optimizer, opt_idx):
        # unfreeze any module you want by overriding this function

        # When ``current_epoch`` is 5, backbone will start to be trained.
        if current_epoch == self._unfreeze_epoch:
            self.unfreeze_and_add_param_group(
                pl_module.backbone,
                optimizer,
            )


# Pass the callback to trainer.finetune
trainer.finetune(model, datamodule, strategy=FeatureExtractorFreezeUnfreeze(unfreeze_epoch=5))